# Correlation function of DR72 SDSS VAGC Catalog

First import all the modules such as healpy and astropy needed for analyzing the structure

In [1]:
import healpix_util as hu
import astropy as ap
import numpy as np
from astropy.io import fits
from astropy.table import Table
import astropy.io.ascii as ascii
from astropy.io import fits
from astropy.constants import c
import matplotlib.pyplot as plt
import math as m
from math import pi
import scipy.special as sp
from scipy import integrate
import warnings
from sklearn.neighbors import BallTree
import pickle
import pymangle
from scipy.optimize import curve_fit
%matplotlib inline

In [ ]:
dr7full=ascii.read("./input/DR7-Full.ascii")
dr7full

In [ ]:
z=dr7full['col3']
rad=dr7full['col1']
decd=dr7full['col2']

In [28]:
#Ez = lambda x: 1.0/m.sqrt(0.3*(1+x)**3+0.7)
Om=0.25
Ol=0.75
Ok=0.0
def Ez(zv):
    return 1.0/m.sqrt(Om*(1.0+zv)**3+Ok*(1.0+zv)**2+Ol)

np.vectorize(Ez)
#Calculate comoving distance of a data point using the Redshift - This definition is based on the cosmology model we take. Here the distance for E-dS universe is considered. Also note that c/H0 ratio is cancelled in the equations and hence not taken.

In [29]:
def DC_LCDM(z):
  return integrate.quad(Ez, 0, z)[0]
DC_LCDM=np.vectorize(DC_LCDM)

In [30]:
DC_LCDM(2.0)

array(1.2660810187999292)

In [ ]:
DC=DC_LCDM(z)
DC

In [ ]:
dr7f = open("./output/DR72srarflcdm1.dat",'w')
dr7f.write("z\t ra\t dec\t s\t rar\t decr \n")

for i in range(0,len(dr7full)):
    dr7f.write("%f\t " %z[i])
    dr7f.write("%f\t %f\t " %(rad[i],decd[i]))
    dr7f.write("%f\t " %DC[i])
    dr7f.write("%f\t %f\n " %(rad[i]*pi/180.0,decd[i]*pi/180.0))
dr7f.close()

In [6]:
data=ascii.read("./output/DR72srarflcdm1.dat")

In [ ]:
data['z']

In [ ]:
data['s']

In [ ]:
data['rar']

In [ ]:
data['decr']

In [ ]:
NSIDE=512
dr72hpix=hu.HealPix("ring",NSIDE)

In [ ]:
pixdata = open("./output/pixdatadr72VAGCfulllcdm1.dat",'w')
pixdata.write("z\t pix \n")

for i in range(0,len(data)):
    pixdata.write("%f\t" %data['z'][i])
    pixdata.write("%d\n" %dr72hpix.eq2pix(data['ra'][i],data['dec'][i]))
pixdata.close()

In [ ]:
pixdata = ascii.read("./output/pixdatadr72VAGCfulllcdm1.dat")
hpixdata=np.array(np.zeros(hu.nside2npix(NSIDE)))
for j in range(len(pixdata)):
    hpixdata[pixdata[j]['pix']]+=1

In [ ]:
hpixdata

In [ ]:
hu.mollview(hpixdata,rot=180)

In [ ]:
mangle=pymangle.Mangle("./masks/")

Ref: https://pypi.python.org/pypi/pymangle/

In [ ]:
%%time
rar,decr=mangle.genrand(2*len(data))

In [22]:
dataR=ascii.read("./output/rand200kdr72.dat")

In [23]:
dataR['z']

0.450184
0.367165
0.413324
0.32247
0.322646
0.233469
0.313645
0.351716
0.403208
0.37691
0.168714


In [24]:
dataR['ra']

171.460447
156.861634
135.655956
151.1135
327.045321
222.706587
132.163348
214.239217
310.627283
143.795051
168.941476


In [25]:
dataR['dec']

14.210816
40.750823
19.592957
24.245002
-7.737766
21.927202
15.281744
5.973424
1.01386
15.993848
43.319462


In [31]:
DCr=DC_LCDM(dataR['z'])

In [32]:
rdr7f = open("./output/rDR72srarflcdm1.dat",'w')
rdr7f.write("z\t ra\t dec\t s\t rar\t decr \n")

for i in range(0,len(dataR)):
    rdr7f.write("%f\t " %dataR['z'][i])
    rdr7f.write("%f\t %f\t " %(dataR['ra'][i],dataR['dec'][i]))
    rdr7f.write("%f\t " %DCr[i])
    rdr7f.write("%f\t %f\n " %(dataR['ra'][i]*pi/180.0,dataR['dec'][i]*pi/180.0))
rdr7f.close()

In [ ]:
r=ascii.read("./output/rDR7200kLCsrarf.dat")

In [ ]:
rar

In [ ]:
decr

In [ ]:
zr=np.array([data['z'],data['z']])

In [ ]:
zr

In [ ]:
zr=zr.flatten()

In [ ]:
zr

In [ ]:
print len(zr)
print len(decr)

In [ ]:
rdr7f = open("./output/rDR72srarf.dat",'w')
rdr7f.write("z\t ra\t dec\t s\t rar\t decr \n")

for i in range(0,len(zr)):
    rdr7f.write("%f\t " %zr[i])
    rdr7f.write("%f\t %f\t " %(rar[i]*180.0/pi,decr[i]*180.0/pi))
    rdr7f.write("%f\t " %DC_LCDM(zr[i]))
    rdr7f.write("%f\t %f\n " %(rar[i],decr[i]))
rdr7f.close()

In [ ]:
dataR=ascii.read("./output/rDR72srarf.dat")

In [ ]:
dataR['z']

In [2]:
dat=ascii.read("./output/DR72srarflcdm1.dat")

In [3]:
dat=np.array([dat['s'],dat['rar'],dat['decr']])

In [7]:
dat.reshape(3,len(data['s']))

array([[ 0.408923,  0.33995 ,  0.378663, ...,  0.340754,  0.211635,
         0.322813],
       [ 0.980185,  1.016617,  0.950251, ...,  2.830242,  2.831786,
         2.838233],
       [-0.003863,  0.003776,  0.010821, ...,  0.172112,  0.173661,
         0.173055]])

In [8]:
dat=dat.transpose()

In [9]:
dat

array([[ 0.408923,  0.980185, -0.003863],
       [ 0.33995 ,  1.016617,  0.003776],
       [ 0.378663,  0.950251,  0.010821],
       ..., 
       [ 0.340754,  2.830242,  0.172112],
       [ 0.211635,  2.831786,  0.173661],
       [ 0.322813,  2.838233,  0.173055]])

In [10]:
from lcdmmetric import *

In [11]:
LCDMmetricsq(dat[0],dat[1])

0.004949874769097584

In [12]:
bins=np.arange(0.005,0.08,0.0025)

In [13]:
print bins

[ 0.005   0.0075  0.01    0.0125  0.015   0.0175  0.02    0.0225  0.025
  0.0275  0.03    0.0325  0.035   0.0375  0.04    0.0425  0.045   0.0475
  0.05    0.0525  0.055   0.0575  0.06    0.0625  0.065   0.0675  0.07
  0.0725  0.075   0.0775]


In [14]:
Nbins=len(bins)

In [15]:
Nbins

30

In [ ]:
help(BallTree)

In [16]:
%%time
BT_D = BallTree(dat,metric='pyfunc',func=LCDMmetricsq,leaf_size=15) 

with open('BTDdr72lcdm1.pkl', 'w') as f:
    pickle.dump(BT_D,f)

CPU times: user 4.84 s, sys: 46.5 ms, total: 4.89 s
Wall time: 4.94 s


In [17]:
with open('./BTDdr72lcdm1.pkl') as f:
    BTD = pickle.load(f)
    
BTD

In [18]:
binsq=bins**2
binsq

array([  2.50000000e-05,   5.62500000e-05,   1.00000000e-04,
         1.56250000e-04,   2.25000000e-04,   3.06250000e-04,
         4.00000000e-04,   5.06250000e-04,   6.25000000e-04,
         7.56250000e-04,   9.00000000e-04,   1.05625000e-03,
         1.22500000e-03,   1.40625000e-03,   1.60000000e-03,
         1.80625000e-03,   2.02500000e-03,   2.25625000e-03,
         2.50000000e-03,   2.75625000e-03,   3.02500000e-03,
         3.30625000e-03,   3.60000000e-03,   3.90625000e-03,
         4.22500000e-03,   4.55625000e-03,   4.90000000e-03,
         5.25625000e-03,   5.62500000e-03,   6.00625000e-03])

In [19]:
%%time
counts_DD=BTD.two_point_correlation(dat,binsq)
print counts_DD

[   398144    762387   1339274   2191788   3382758   4978021   7042612
   9632016  12827932  16684648  21279983  26677220  32962485  40179458
  48371637  57601351  67921520  79370882  92002867 105815985 120862717
 137188043 154832230 173812300 194093787 215786834 238874784 263357865
 289309916 316747614]
CPU times: user 24min 27s, sys: 4.02 s, total: 24min 31s
Wall time: 24min 35s


In [20]:
with open('./DDDr72lcdm1.pkl', 'w') as f:
    pickle.dump(counts_DD,f)

with open('./DDDr72lcdm1.pkl') as f:
    counts_DD = pickle.load(f)
    
counts_DD

array([   398144,    762387,   1339274,   2191788,   3382758,   4978021,
         7042612,   9632016,  12827932,  16684648,  21279983,  26677220,
        32962485,  40179458,  48371637,  57601351,  67921520,  79370882,
        92002867, 105815985, 120862717, 137188043, 154832230, 173812300,
       194093787, 215786834, 238874784, 263357865, 289309916, 316747614])

In [21]:
DD=np.diff(counts_DD)
DD

array([  364243,   576887,   852514,  1190970,  1595263,  2064591,
        2589404,  3195916,  3856716,  4595335,  5397237,  6285265,
        7216973,  8192179,  9229714, 10320169, 11449362, 12631985,
       13813118, 15046732, 16325326, 17644187, 18980070, 20281487,
       21693047, 23087950, 24483081, 25952051, 27437698])

In [33]:
dataR=ascii.read("./output/rDR72srarflcdm1.dat")

In [34]:
dataR['z']

0.450184
0.367165
0.413324
0.32247
0.322646
0.233469
0.313645
0.351716
0.403208
0.37691
0.168714


In [35]:
datR=np.array([dataR['s'],dataR['rar'],dataR['decr']])

In [36]:
datR.reshape(3,len(dataR['s']))

array([[ 0.408923,  0.33995 ,  0.378662, ...,  0.338265,  0.30569 ,
         0.198644],
       [ 2.992549,  2.737752,  2.367643, ...,  3.228052,  3.218296,
         4.056418],
       [ 0.248026,  0.711236,  0.341962, ...,  0.199013,  0.052979,
         0.170602]])

In [37]:
datR=datR.transpose()

In [38]:
datR

array([[ 0.408923,  2.992549,  0.248026],
       [ 0.33995 ,  2.737752,  0.711236],
       [ 0.378662,  2.367643,  0.341962],
       ..., 
       [ 0.338265,  3.228052,  0.199013],
       [ 0.30569 ,  3.218296,  0.052979],
       [ 0.198644,  4.056418,  0.170602]])

In [39]:
from lcdmmetric import *

In [40]:
LCDMmetricsq(datR[0],datR[1])

0.040646864133247085

In [34]:
bins=np.arange(0.005,0.08,0.0025)

In [35]:
print bins

[ 0.005   0.0075  0.01    0.0125  0.015   0.0175  0.02    0.0225  0.025
  0.0275  0.03    0.0325  0.035   0.0375  0.04    0.0425  0.045   0.0475
  0.05    0.0525  0.055   0.0575  0.06    0.0625  0.065   0.0675  0.07
  0.0725  0.075   0.0775]


In [36]:
Nbins=len(bins)

In [37]:
Nbins

30

In [38]:
help(BallTree)

Help on class BallTree in module sklearn.neighbors.ball_tree:

class BallTree(BinaryTree)
 |  BallTree for fast generalized N-point problems
 |  
 |  BallTree(X, leaf_size=40, metric='minkowski', \**kwargs)
 |  
 |  Parameters
 |  ----------
 |  X : array-like, shape = [n_samples, n_features]
 |      n_samples is the number of points in the data set, and
 |      n_features is the dimension of the parameter space.
 |      Note: if X is a C-contiguous array of doubles then data will
 |      not be copied. Otherwise, an internal copy will be made.
 |  
 |  leaf_size : positive integer (default = 40)
 |      Number of points at which to switch to brute-force. Changing
 |      leaf_size will not affect the results of a query, but can
 |      significantly impact the speed of a query and the memory required
 |      to store the constructed tree.  The amount of memory needed to
 |      store the tree scales as approximately n_samples / leaf_size.
 |      For a specified ``leaf_size``, a leaf 

In [41]:
%%time
BT_R = BallTree(dat,metric='pyfunc',func=LCDMmetricsq,leaf_size=15) 

with open('BTRdr72lcdm1.pkl', 'w') as f:
    pickle.dump(BT_R,f)

CPU times: user 4.69 s, sys: 60.5 ms, total: 4.75 s
Wall time: 4.85 s


In [42]:
with open('./BTRdr72lcdm1.pkl') as f:
    BTR = pickle.load(f)
    
BTR

In [41]:
binsq=bins**2
binsq

array([  2.50000000e-05,   5.62500000e-05,   1.00000000e-04,
         1.56250000e-04,   2.25000000e-04,   3.06250000e-04,
         4.00000000e-04,   5.06250000e-04,   6.25000000e-04,
         7.56250000e-04,   9.00000000e-04,   1.05625000e-03,
         1.22500000e-03,   1.40625000e-03,   1.60000000e-03,
         1.80625000e-03,   2.02500000e-03,   2.25625000e-03,
         2.50000000e-03,   2.75625000e-03,   3.02500000e-03,
         3.30625000e-03,   3.60000000e-03,   3.90625000e-03,
         4.22500000e-03,   4.55625000e-03,   4.90000000e-03,
         5.25625000e-03,   5.62500000e-03,   6.00625000e-03])

In [43]:
%%time
counts_RR=BTR.two_point_correlation(datR,binsq)
print counts_RR

[   216135    713523   1654307   3164595   5367372   8379495  12324259
  17325453  23511761  31003414  39944117  50461801  62664671  76693702
  92653499 110647492 130760394 153091981 177712818 204702751 234124818
 266025681 300449876 337484622 377175469 419547944 464657335 512494963
 563128541 616621606]
CPU times: user 46min 44s, sys: 6.39 s, total: 46min 51s
Wall time: 46min 56s


In [44]:
with open('./RRDr72lcdm1.pkl', 'w') as f:
    pickle.dump(counts_RR,f)

with open('./RRDr72lcdm1.pkl') as f:
    counts_RR = pickle.load(f)
    
counts_RR

array([   216135,    713523,   1654307,   3164595,   5367372,   8379495,
        12324259,  17325453,  23511761,  31003414,  39944117,  50461801,
        62664671,  76693702,  92653499, 110647492, 130760394, 153091981,
       177712818, 204702751, 234124818, 266025681, 300449876, 337484622,
       377175469, 419547944, 464657335, 512494963, 563128541, 616621606])

In [45]:
RR=np.diff(counts_RR)
RR

array([  497388,   940784,  1510288,  2202777,  3012123,  3944764,
        5001194,  6186308,  7491653,  8940703, 10517684, 12202870,
       14029031, 15959797, 17993993, 20112902, 22331587, 24620837,
       26989933, 29422067, 31900863, 34424195, 37034746, 39690847,
       42372475, 45109391, 47837628, 50633578, 53493065])

In [47]:
%%time
counts_DR=BTR.two_point_correlation(dat,binsq)
print counts_DR

[   398144    762387   1339274   2191788   3382758   4978021   7042612
   9632016  12827932  16684648  21279983  26677220  32962485  40179458
  48371637  57601351  67921520  79370882  92002867 105815985 120862717
 137188043 154832230 173812300 194093787 215786834 238874784 263357865
 289309916 316747614]
CPU times: user 22min 46s, sys: 2.47 s, total: 22min 49s
Wall time: 31min 59s


In [48]:
with open('./DRDr72lcdm1.pkl', 'w') as f:
    pickle.dump(counts_DR,f)

with open('./DRDr72lcdm1.pkl') as f:
    counts_DR = pickle.load(f)
    
counts_DR

array([   398144,    762387,   1339274,   2191788,   3382758,   4978021,
         7042612,   9632016,  12827932,  16684648,  21279983,  26677220,
        32962485,  40179458,  48371637,  57601351,  67921520,  79370882,
        92002867, 105815985, 120862717, 137188043, 154832230, 173812300,
       194093787, 215786834, 238874784, 263357865, 289309916, 316747614])

In [49]:
DR=np.diff(counts_DR)
DR

array([  364243,   576887,   852514,  1190970,  1595263,  2064591,
        2589404,  3195916,  3856716,  4595335,  5397237,  6285265,
        7216973,  8192179,  9229714, 10320169, 11449362, 12631985,
       13813118, 15046732, 16325326, 17644187, 18980070, 20281487,
       21693047, 23087950, 24483081, 25952051, 27437698])

In [50]:
from __future__ import division
correl=(4.0*DD-4.0*DR+1.0*RR)/(1.0*RR)

In [52]:
from operator import truediv
correl=truediv((4.0*DD-4.0*DR+1.0*RR),(1.0*RR))

In [53]:
correl

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [54]:
4.0*DD-4.0*DR+1.0*RR

array([   497388.,    940784.,   1510288.,   2202777.,   3012123.,
         3944764.,   5001194.,   6186308.,   7491653.,   8940703.,
        10517684.,  12202870.,  14029031.,  15959797.,  17993993.,
        20112902.,  22331587.,  24620837.,  26989933.,  29422067.,
        31900863.,  34424195.,  37034746.,  39690847.,  42372475.,
        45109391.,  47837628.,  50633578.,  53493065.])

In [55]:
RR

array([  497388,   940784,  1510288,  2202777,  3012123,  3944764,
        5001194,  6186308,  7491653,  8940703, 10517684, 12202870,
       14029031, 15959797, 17993993, 20112902, 22331587, 24620837,
       26989933, 29422067, 31900863, 34424195, 37034746, 39690847,
       42372475, 45109391, 47837628, 50633578, 53493065])

In [58]:
DR

array([  364243,   576887,   852514,  1190970,  1595263,  2064591,
        2589404,  3195916,  3856716,  4595335,  5397237,  6285265,
        7216973,  8192179,  9229714, 10320169, 11449362, 12631985,
       13813118, 15046732, 16325326, 17644187, 18980070, 20281487,
       21693047, 23087950, 24483081, 25952051, 27437698])

In [57]:
DD = np.array([  364243,   576887,   852514,  1190970,  1595263,  2064591,
        2589404,  3195916,  3856716,  4595335,  5397237,  6285265,
        7216973,  8192179,  9229714, 10320169, 11449362, 12631985,
       13813118, 15046732, 16325326, 17644187, 18980070, 20281487,
       21693047, 23087950, 24483081, 25952051, 27437698])

In [ ]:
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

In [ ]:
from lcdmmetric import *

In [ ]:
z=np.array(data['z'])

In [ ]:
zkde=z.reshape(1,-1)

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(zkde)

In [ ]:
kde

In [ ]:
X_plot = np.arange(z.min(), z.max(), z.size())[:, np.newaxis]

In [ ]:
log_dens = kde.score_samples(zkde)

In [ ]:
log_dens

In [ ]:
d=ascii.read("./output/DR72LCsrarf.dat")

In [ ]:
d

In [ ]:
dataR=ascii.read("./output/rand200kdr72.dat")

In [ ]:
dataR['z']

In [ ]:
dataR['ra']

In [ ]:
dataR['dec']

In [ ]:
DCLCR=DC_LC(dataR['z'])

In [ ]:
rdr7f = open("./output/rDR7200kLCsrarf.dat",'w')
rdr7f.write("z\t ra\t dec\t s\t rar\t decr \n")

for i in range(0,len(dataR)):
    rdr7f.write("%f\t " %dataR['z'][i])
    rdr7f.write("%f\t %f\t " %(dataR['ra'][i],dataR['dec'][i]))
    rdr7f.write("%f\t " %DCLCR[i])
    rdr7f.write("%f\t %f\n " %(dataR['ra'][i]*pi/180.0,dataR['dec'][i]*pi/180.0))
rdr7f.close()

In [ ]:
r=ascii.read("./output/rDR7200kLCsrarf.dat")

In [ ]:
r

In [ ]:
dr7fdat=ascii.read("./output/DR7srarf.dat")

In [ ]:
dr7fdat['s'][1:300]

In [ ]:
#fdata=fits.open("/Users/rohin/Downloads/DR7-Full.fits")

In [ ]:
#fdata.writeto("./output/DR7fulltrim.fits")

In [ ]:
fdata=fits.open("./output/DR7fulltrim.fits")

In [ ]:
cols=fdata[1].columns

In [ ]:
cols.del_col('ZTYPE')

In [ ]:
cols.del_col('SECTOR')
cols.del_col('FGOTMAIN')
cols.del_col('QUALITY')
cols.del_col('ISBAD')
cols.del_col('M')
cols.del_col('MMAX')
cols.del_col('ILSS')
cols.del_col('ICOMB')
cols.del_col('VAGC_SELECT')
cols.del_col('LSS_INDEX')
cols.del_col('FIBERWEIGHT')
cols.del_col('PRIMTARGET')
cols.del_col('MG')
cols.del_col('SECTOR_COMPLETENESS')
cols.del_col('COMOV_DENSITY')
cols.del_col('RADIAL_WEIGHT')

In [ ]:
fdata[1].columns

In [ ]:
fdata.writeto("./output/DR7fullzradec.fits")

In [ ]:
fdat=fits.open("./output/DR7fullzradec.fits")

In [ ]:
fdat[1].columns

In [ ]:
fdat[1].data['Z']

In [ ]:
fdat[1].data['RA']

In [ ]:
comovlcdm=DC_LCDM(fdat[1].data['Z'])

In [ ]:
fdat[1].data['Z']

In [ ]:
comovlcdm

In [ ]:
comovlcdm.dtype

In [ ]:
#cols=fdat[1].columns

In [ ]:
nc=fits.Column(name='COMOV',format='D',array=comovlcdm)

In [ ]:
nc1=fits.Column(name='COMOV',format='D')

In [ ]:
fdata[1].data['Z']

In [ ]:
fdata[1].data['RA']

In [ ]:
nc

In [ ]:
nc.dtype

In [ ]:
#cols.add_col(nc)

In [ ]:
fdat[1].columns

In [ ]:
fdat[1].columns.info()

In [ ]:
fdat[1].columns.add_col(nc1)

In [ ]:
fdat[1].data['COMOV']=comovlcdm

In [ ]:
comovlcdm

In [ ]:
fdat[1].data['Z']

In [ ]:
fdat[1].data['COMOV']

In [ ]:
fdat[1].data['RA']

In [ ]:
fdat[1].data['RA']=fdat[1].data['RA']*pi/180.0

In [ ]:
comovlcdm=DC_LCDM(fdat[1].data['Z'])
comovlcdm

Random catalog created based on the survey limitations also taken from http://cosmo.nyu.edu/~eak306/SDSS-LRG.html

In [ ]:
dataR=fits.open("/Users/rohin/Downloads/random-DR7-Full.fits")

In [ ]:
dataR

In [ ]:
dataR=dataR[1].data

In [ ]:
len(dataR)

In [ ]:
NSIDE=512
dr72hpix=hu.HealPix("ring",NSIDE)

In [ ]:
pixdata = open("./output/pixdatadr72VAGCfullrand.dat",'w')
pixdata.write("z\t pix \n")

for i in range(0,len(data)-1):
    pixdata.write("%f\t" %data['z'][i])
    pixdata.write("%d\n" %dr72hpix.eq2pix(dataR['ra'][i],dataR['dec'][i]))
pixdata.close()

In [ ]:
pixdata = ascii.read("./output/pixdatadr72VAGCfullrand.dat")
hpixdata=np.array(np.zeros(hu.nside2npix(NSIDE)))
for j in range(len(pixdata)):
    hpixdata[pixdata[j]['pix']]+=1

In [ ]:
hpixdata

In [ ]:
hu.mollview(hpixdata,rot=180)

In [ ]:
hu.orthview(hpixdata)